In [ ]:
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [ ]:
# import data.tsv file and create a dataframe

df = pd.read_csv('data.tsv', sep='\t')

# split df into training and test sets

train_df = df.sample(frac=0.8, random_state=0)
test_df = df.drop(train_df.index)

train_x = train_df.drop(0, axis=1)
train_y = train_df[0]

test_x = test_df.drop(0, axis=1)
test_y = test_df[0]

# train a model pytorch recurrent neural network



class SitUpDetector(nn.Module):
    def __init__(self):
        super(SitUpDetector, self).__init__()
        self.fc1 = nn.Linear(10, 100)
        self.fc2 = nn.Linear(100, 100)
        self.fc3 = nn.Linear(100, 1)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = torch.sigmoid(self.fc3(x))
        return x

model = SitUpDetector()
criterion = nn.BCELoss()
optimizer = optim.SGD(model.parameters(), lr=0.01)

for epoch in range(1000):
    optimizer.zero_grad()
    output = model(train_x)
    loss = criterion(output, train_y)
    loss.backward()
    optimizer.step()
    if epoch % 100 == 0:
        print(loss.item())

# test the model

with torch.no_grad():
    output = model(test_x)
    print(output)
    print(test_y)

# save the model

torch.save(model.state_dict(), 'sit_up_detector.pth')




